# Neural Network

![](img/nn and backpropagation.png)

## Algorithm

### Stocastic Gradient Descent
1. Randomly shuffle the training set
2. Segment the training set into mini batches
3. For each mini batch:
    - Update the network's weights and biases using the partial derivatives of weights and biases

### Update network
1. For each training example in the mini batch:
    - Apply backpropagation

### Backpropagation
1. Feedforward to calculate zs and activations
2. Backward pass to calculate deltas
3. Get the partial derivatives of weights and biases using the calculated deltas

In [2]:
import numpy as np
import random

In [3]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [4]:
class Network():
    """
    The neural network class
    """
    
    def __init__(self, sizes):
        """
        sizes: an array containing the number of neuons in each layer, layer 0 is the input layer
        num_layer: number of layers in the network
        biases: randomly initialized array containing column vector of biases for each layer
        weights: randomly initialized array containig m(l-1) * m(l) weight matrix for each layer
        """
        self.num_layer = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[1:], sizes[:-1])]
    
    def feedforward(self, a):
        """
        Feed forward the network and get zs and activations
        Activations in the final layer is the output
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w.T, a) + b)
        return a
    
    def sgd(self, training_data, epochs, mini_batch_size, alpha, test_data=None):
        """
        Mini-batch stocastic gradient descent
        training_data: input training data
        epochs: number of times to run the whole training data
        mini_batch_size: how many training examples in a mini batch
        alpha: learning rate
        test_data: if provided, evaluate performance after each epoch
        """
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
            
        training_data = list(training_data)
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_network(mini_batch, alpha)
            
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
    
    def update_network(self, mini_batch, alpha):
        """
        Update the weights and biases of the network
        mini_batch: mini batch containg training examples
        alpha: learning rate
        """
        batch_deriv_b = [np.zeros(b.shape) for b in self.biases]
        batch_deriv_w = [np.zeros(w.shape) for w in self.weights]
        
        # Get the sum of derivatives of the mini batch
        for x, y in mini_batch:
            this_deriv_w, this_deriv_b = self.backprop(x, y)
            batch_deriv_b = [nb + dnb for nb, dnb in zip(batch_deriv_b, this_deriv_b)]
            batch_deriv_w = [nw + dnw for nw, dnw in zip(batch_deriv_w, this_deriv_w)]
        
        # Update weights and biases
        self.weights = [w - (alpha / len(mini_batch)) * nw for w, nw in zip(self.weights, batch_deriv_w)]
        self.biases = [b - (alpha / len(mini_batch)) * nb for b, nb in zip(self.biases, batch_deriv_b)]
        
    def backprop(self, x, y):
        """
        Backpropagate to calculate cost function gradients for each training example
        x, y: single input training example
        """
        deriv_b = [np.zeros(b.shape) for b in self.biases]
        deriv_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward to calculate zs and activations
        activations = [x]
        zs = []
        
        for w, b in zip(self.weights, self.biases):
            zs.append(np.dot(w.T, activations[-1]) + b)
            activations.append(sigmoid(zs[-1]))
        
        # Backward pass
        # Output layer delta and gradients
        delta = sigmoid_prime(zs[-1]) * self.cost_derivative(activations[-1], y)
        deriv_b[-1] = delta
        deriv_w[-1] = np.dot(activations[-2], delta.T)
        
        # Previous layers
        for l in range(2, self.num_layer):
            z = zs[-l]
            delta = np.dot(self.weights[-l + 1], delta) * sigmoid_prime(z)
            deriv_b[-l] = delta
            deriv_w[-l] = np.dot(activations[-l - 1], delta.T)
            
        return (deriv_w, deriv_b)
        
    def evaluate(self, test_data):
        """
        Return the number of correctly predicted results
        test_data: test dataset
        test_res: test result, tuples of predicted digit and the real digit
        """
        test_res = [(np.argmax(self.feedforward(x)), y) for x, y in test_data]
        return sum(int(pred == y) for pred, y in test_res)
    
    def cost_derivative(self, output_activations, y):
        """
        Return the derivative of the cost function
        """
        return output_activations - y
    
def sigmoid(x):
    """The sigmoid function"""
    return 1/(1 + np.exp(-x))

def sigmoid_prime(x):
    """Derivative of the sigmoid function"""
    return sigmoid(x) * (1 - sigmoid(x))

In [5]:
# Test
# Layers: 784, 30, 10
net = Network([784, 30, 10])
net.sgd(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8379 / 10000
Epoch 1: 8483 / 10000
Epoch 2: 9185 / 10000
Epoch 3: 9321 / 10000
Epoch 4: 9344 / 10000
Epoch 5: 9401 / 10000
Epoch 6: 9410 / 10000
Epoch 7: 9417 / 10000
Epoch 8: 9436 / 10000
Epoch 9: 9458 / 10000
Epoch 10: 9462 / 10000
Epoch 11: 9436 / 10000
Epoch 12: 9467 / 10000
Epoch 13: 9460 / 10000
Epoch 14: 9492 / 10000
Epoch 15: 9509 / 10000
Epoch 16: 9517 / 10000
Epoch 17: 9476 / 10000
Epoch 18: 9499 / 10000
Epoch 19: 9500 / 10000
Epoch 20: 9484 / 10000
Epoch 21: 9487 / 10000
Epoch 22: 9513 / 10000
Epoch 23: 9503 / 10000
Epoch 24: 9491 / 10000
Epoch 25: 9497 / 10000
Epoch 26: 9484 / 10000
Epoch 27: 9500 / 10000
Epoch 28: 9507 / 10000
Epoch 29: 9502 / 10000
